In [1]:
import pandas as pd
import numpy as np
import os
import time
from IPython.display import clear_output

In [2]:
root = '/home/MIBS/facenet'
target_folder = os.path.join(root, 'embedding', 'similarity')

## Read data

In [86]:
df_sim = pd.read_csv(os.path.join(target_folder, 'similarity_tmp2.csv'), index_col=0)
df_sim.head()

,1000046113_190146.jpg,1000046113_190147.jpg,1000046113_190148.jpg,1000046113_190149.jpg,1000046113_190150.jpg
1000046113_190136.jpg,0.082246,0.080399,0.096319,0.088590,0.068485
1000046113_190137.jpg,0.055225,0.058025,0.085810,0.073788,0.043859
1000046113_190138.jpg,0.061048,0.062942,0.078878,0.075144,0.047839
1000046113_190139.jpg,0.061858,0.072317,0.086135,0.084699,0.051873
1000046113_190140.jpg,0.048721,0.052151,0.065121,0.063749,0.042856


In [87]:
df_sim

,1000046113_190146.jpg,1000046113_190147.jpg,1000046113_190148.jpg,1000046113_190149.jpg,1000046113_190150.jpg
1000046113_190136.jpg,0.082246,0.080399,0.096319,0.088590,0.068485
1000046113_190137.jpg,0.055225,0.058025,0.085810,0.073788,0.043859
1000046113_190138.jpg,0.061048,0.062942,0.078878,0.075144,0.047839
1000046113_190139.jpg,0.061858,0.072317,0.086135,0.084699,0.051873
1000046113_190140.jpg,0.048721,0.052151,0.065121,0.063749,0.042856
...,...,...,...,...,...
9876_10089.jpg,0.886060,0.890838,0.858810,0.885430,0.869469
9876_10090.jpg,0.866173,0.871467,0.837671,0.869012,0.853637
9876_10091.jpg,0.851032,0.856232,0.822264,0.851635,0.836307
9876_10092.jpg,0.865678,0.872707,0.841235,0.869336,0.853115


## Check similarity values

In [88]:
similarity_data = []
threshold = .2

tp, tn, fp, fn = 0, 0, 0, 0

for column, data_column in df_sim.items():
    idx_1 = column.split('.')[0].split('_')[0]
    tmp_dict = {column: {'same': {'min': None, 'max': None}, 'different': {'min': None, 'max': None}}}
    for row, value in data_column.items():
        idx_2 = row.split('.')[0].split('_')[0]
        if idx_1 == idx_2:
            # same person
            if value < threshold:
                tp += 1
            else:
                fn += 1
            current_min = tmp_dict[column]['same']['min']
            current_max = tmp_dict[column]['same']['max']
            if current_min == None or current_max == None:
                tmp_dict[column]['same']['min'] = value
                tmp_dict[column]['same']['max'] = value
            else:
                if value < tmp_dict[column]['same']['min']:
                    tmp_dict[column]['same']['min'] = value
                if value > tmp_dict[column]['same']['max']:
                    tmp_dict[column]['same']['max'] = value
        else:
            # different person
            if value < threshold:
                fp += 1
            else:
                tn += 1
            current_min = tmp_dict[column]['different']['min']
            current_max = tmp_dict[column]['different']['max']
            if current_min == None or current_max == None:
                tmp_dict[column]['different']['min'] = value
                tmp_dict[column]['different']['max'] = value
            else:
                if value < tmp_dict[column]['different']['min']:
                    tmp_dict[column]['different']['min'] = value
                if value > tmp_dict[column]['different']['max']:
                    tmp_dict[column]['different']['max'] = value
    similarity_data.append(tmp_dict)

In [89]:
print('TP: {}, TN: {}, FP: {}, FN: {}'.format(tp, tn, fp, fn))

TP: 1708, TN: 2323560, FP: 0, FN: 797


In [90]:
precision = float(tp) / (tp + fp)
recall = float(tp) / (tp + fn)
f1 = float(2 * precision * recall) / (precision + recall)

print('Precision: {} - Recall: {} - F1: {}'.format(precision, recall, f1))

Precision: 1.0 - Recall: 0.6818363273453094 - F1: 0.8108236411108474


In [83]:
for idx, similarity in enumerate(similarity_data):
    reform = {(outerKey, innerKey): values for outerKey, innerDict in similarity.items() for innerKey, values in innerDict.items()}
    if idx == 0:
        tmp_df = pd.DataFrame(reform).T
    else:
        tmp_df = pd.concat([tmp_df, pd.DataFrame(reform).T])

In [84]:
tmp_df

min       max
1000046113_190141.jpg same      -0.000000  0.538949
                      different  0.223408  1.374303
1000046113_190142.jpg same       0.000000  0.548512
                      different  0.206215  1.337139
1000046113_190143.jpg same       0.000000  0.551962
                      different  0.223028  1.316486
1000046113_190144.jpg same       0.000000  0.473234
                      different  0.224620  1.267496
1000046113_190145.jpg same       0.000000  0.541924
                      different  0.279706  1.410728

In [85]:
tmp_df.describe()

,min,max
count,10.000000,10.000000
mean,0.115698,0.936073
std,0.123382,0.429182
min,-0.000000,0.473234
25%,0.000000,0.543571
50%,0.103108,0.909729
75%,0.223313,1.331976
max,0.279706,1.410728
